In [1]:
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import argparse
from tqdm import tqdm  # optional progress bar
import pandas as pd
import torch
from torch import nn
from torch import nn, optim
from torch.nn import functional as F
from collections import OrderedDict
from torch import Tensor
import pprint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

hyperparams = {
    "num_epochs": 1,
    "batch_size": 100,
    "learning_rate": 0.001,
}

cuda


In [2]:
def cells_to_id(cells):
    cti = dict()
    for i, cell in enumerate(cells):
        cti[cell] = i
    return cti

train_cells = ['E065', 'E004', 'E066', 'E005', 'E012', 'E027', 'E053', 'E013', 'E028', 'E061', 'E109', 'E120', 'E062', 'E037', 'E038', 'E024', 'E105', 'E011', 'E106', 'E082', 'E097', 'E116', 'E098', 'E058',
               'E117', 'E059', 'E070', 'E118', 'E085', 'E104', 'E119', 'E006', 'E127', 'E047', 'E094', 'E007', 'E054', 'E128', 'E095', 'E055', 'E114', 'E100', 'E056', 'E016', 'E122', 'E057', 'E123', 'E079', 'E003', 'E050']

train_cells_dict = cells_to_id(train_cells)

eval_cells = ['E065', 'E004', 'E066', 'E005', 'E012', 'E027', 'E053', 'E013', 'E028', 'E061', 'E109', 'E120', 'E062', 'E037', 'E038', 'E024', 'E071', 'E105', 'E087', 'E011', 'E106', 'E096', 'E082', 'E097',
              'E116', 'E098', 'E058', 'E117', 'E084', 'E059', 'E070', 'E118', 'E085', 'E104', 'E119', 'E006', 'E112', 'E127', 'E047', 'E094', 'E007', 'E054', 'E113', 'E128', 'E095', 'E055', 'E114', 'E100', 'E056', 'E016', 'E122', 'E057', 'E123', 'E079', 'E003', 'E050']

eval_cells_dict = cells_to_id(eval_cells)

def onehot_encode(sequence):
    molecule_to_vec = {
        'A': 0,
        'C': 1,
        'T': 2,
        'G': 3,
        'N': 4
    }
    onehot_matrix = np.zeros((len(sequence), 5))
    for i in range(len(sequence)):
        onehot_matrix[i][molecule_to_vec[sequence[i]]] = 1

    return onehot_matrix

class HistoneDataset(Dataset):
    def __init__(self, input_file, id2seq, cell=None):
        """
        :param input_file: the data file pathname
        """
        self.id2seq = id2seq

        # [50, 16000, 100, 7]
        # [cell_types, genes, bins, (columns)]
        # columns = GeneID, H3K27me3, H3K36me3, H3K4me1, H3K4me3, H3K9me3, Expression Value (same for entire bin)
        # columns 0: GeneId, 1-5: Histone Marks, 6: Expression Value
        npdata = np.load(input_file)
        cell_types = npdata.files

        # [cell_types, genes, bins, histomes]
        input = []
        # [cell_types, genes, expression]
        output = []
        # [cell_types, genes, expression]
        ids = []
        # types
        types = []

        if cell is None:
            for cell in cell_types:
                cell_data = npdata[cell]
                id = cell_data[:, 0, 0]
                hm_data = cell_data[:, :, 1:6]
                exp_values = cell_data[:, 0, 6]
                ids.append(id)
                input.append(hm_data)
                output.append(exp_values)
                type.extend([cell] * cell_data.shape[0])
        else:
            cell_data = npdata[cell]
            id = cell_data[:, 0, 0]
            hm_data = cell_data[:, :, 1:6]
            exp_values = cell_data[:, 0, 6]
            ids.append(id)
            input.append(hm_data)
            output.append(exp_values)
            types.extend([cell] * cell_data.shape[0])

        # [cell_types*genes, bins, histomes]
        input = np.concatenate(input, axis=0)
        # [cell_types*genes, expression]
        output = np.concatenate(output, axis=0)
        ids = np.concatenate(ids, axis=0)

        self.x = []
        self.y = []
        self.id = ids
        self.type = np.asarray(types)

        for x in input:
            self.x.append(torch.tensor(x))

        for y in output:
            self.y.append(torch.tensor(y))

    def __len__(self):
        """
        len should return a the length of the dataset

        :return: an integer length of the dataset
        """
        # TODO: Override method to return length of dataset
        return len(self.y)

    def __getitem__(self, idx):
        """
        getitem should return a tuple or dictionary of the data at some index
        In this case, you should return your original and target sentence and
        anything else you may need in training/validation/testing.

        :param idx: the index for retrieval

        :return: tuple or dictionary of the data
        """
        # TODO: Override method to return the items in dataset
        item = {
            "cell_type": self.type[idx],
            "id": self.id[idx],
            "x": torch.cat((self.x[idx], self.id2seq[self.id[idx]]), dim=0),
            "y": self.y[idx],
        }
        return item

In [3]:
class DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(DenseLayer, self).__init__()
        self.norm1 = nn.BatchNorm2d(num_input_features)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(num_input_features, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)
        self.norm2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
        self.dropout = nn.Dropout(p=drop_rate)

    def bn_function(self, inputs):
        # type: (List[Tensor]) -> Tensor
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))
        return bottleneck_output

    def forward(self, input):
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input

        bottleneck_output = self.bn_function(prev_features)

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        new_features = self.dropout(new_features)
        return new_features


class DenseBlock(nn.ModuleDict):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)


class Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(Transition, self).__init__()
        self.norm = nn.BatchNorm2d(num_input_features)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(num_input_features, num_output_features, kernel_size=1, stride=1, bias=False)
        # prevent output from shrinking
        self.pool = nn.AvgPool2d(kernel_size=2, stride=(2, 1))



class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0.5, num_classes=1, theta=0.5):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(1, num_init_features, kernel_size=7, stride=(2, 1),
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=(2, 1), padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = Transition(num_input_features=num_features,
                                    num_output_features=int(num_features * theta))
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = int(num_features * theta)

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out


def densenet():
    # based on tochvision
    model = DenseNet(4, (3, 3, 3, 3), 16)
    return model


In [4]:
def train(model, train_loader, cell):
    print("starting train")
    model = model.to(device)
    loss_fn = torch.nn.MSELoss(
        size_average=None, reduce=None, reduction='mean')
    optimizer = optim.Adam(model.parameters(), hyperparams['learning_rate'])

    model = model.train()
    losses = []
    for epoch in range(hyperparams['num_epochs']):
        for batch in tqdm(train_loader):
            x = batch['x']
            y = batch['y']
            x = x.unsqueeze(1)
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            y_pred = model(x)

            loss = loss_fn(y_pred.squeeze(1), y)

            loss.backward()  # calculate gradients
            optimizer.step()  # update model weights

            losses.insert(0, float(loss.cpu().item()))
            losses = losses[:100]
        print("avg loss:", np.mean(losses))
        torch.save(model.state_dict(), './model' + cell + '.pt')


def validate(model, validate_loader):
    print("starting validation")
    model = model.to(device)
    loss_fn = torch.nn.MSELoss(
        size_average=None, reduce=None, reduction='mean')

    model = model.eval()
    losses = []

    for batch in tqdm(validate_loader):
        x = batch['x']
        y = batch['y']
        # unsqueeze to create 1 input channel
        x = x.unsqueeze(1)
        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)

        loss = loss_fn(y_pred.squeeze(1), y)

        losses.append(float(loss.cpu().item()))

    print("mean loss:", np.mean(losses))


def test(models, test_loaders, cell_types):
    print("starting test")
    classification = []
    for cell in cell_types:
        test_loader = test_loaders[cell]
        cell_models = []
        if cell in train_cells:
            model = models[train_cells_dict[cell]].to(device)
            model = model.eval()
            cell_models.append(model)
        else:
            for cell in train_cells:
                model = models[train_cells_dict[cell]].to(device)
                model = model.eval()
                cell_models.append(model)

        for batch in tqdm(test_loader):
            x = batch['x']
            id = batch['id']
            cell_type = batch['cell_type']
            x = x.unsqueeze(1)
            x = x.to(device)

            y_preds = None
            if len(cell_models) == 1:
                y_preds = cell_models[0](x).unsqueeze(0)
            else:
                for model in cell_models:
                    if y_preds is None:
                        y_preds = model(x).unsqueeze(0)
                    else:
                        y_preds = torch.cat((y_preds, model(x).unsqueeze(0)))

            y_pred = torch.mean(y_preds, 0)
            # print(y_pred)

            for i in range(y_pred.size()[0]):
                # print(cell_type[i].item(), id[i].item(), y_pred[i].item())
                classification.append(
                    (str(cell_type[i].item()) + "_" + str(int(id[i].item())), str(y_pred[i].cpu().item())))

    df = pd.DataFrame(classification, columns=['id', 'expression'])
    df.to_csv('submission.csv', index=False)

In [ ]:
train_model = True
test_model = False
load_model = False
save_model = True

print("Device", device)

models = dict()
for cell in train_cells:
    model = DenseNet(4, (3, 3, 3), 16)
    models[train_cells_dict[cell]] = model
    
location = './'
test_file = location + 'data/eval.npz'
train_file = location + 'data/train.npz'
seq_file = location + 'data/seq_data.csv'

train_dataset = None
validate_dataset = None
test_dataset = None

print("gathering train data", device)
train_loaders = dict()
validate_loaders = dict()
id2seq = dict()
seqs = pd.read_csv(seq_file, index_col=False, names=["id", "sequence"])
for index, row in seqs.iterrows():
    id2seq[row['id']] = torch.tensor(onehot_encode(row['sequence'])).float()
print("gathered sequences")

if train_model:
    train_datasets = dict()
    validate_datasets = dict()
    for cell in train_cells:
        dataset = HistoneDataset(train_file, id2seq, cell)

        split_amount = int(len(dataset) * 0.95)

        train_dataset, validate_dataset = random_split(
            dataset, (split_amount, len(dataset) - split_amount))

        train_datasets[train_cells_dict[cell]] = train_dataset
        validate_datasets[train_cells_dict[cell]] = validate_dataset
    
    for cell in train_cells:
        train_loader = DataLoader(
            train_datasets[train_cells_dict[cell]], batch_size=hyperparams['batch_size'], shuffle=True
        )
        validate_loader = DataLoader(
            validate_datasets[train_cells_dict[cell]], batch_size=hyperparams['batch_size'], shuffle=True
        )

        train_loaders[train_cells_dict[cell]] = train_loader
        validate_loaders[train_cells_dict[cell]] = validate_loader
print("gathered train")
        
cell_types = None
test_loaders = dict()
if test_model:
    test_datasets = dict()
    npzfile = np.load(test_file)
    cell_types = npzfile.files
    for cell in cell_types:
        assert cell in eval_cells_dict
        test_dataset = HistoneDataset(test_file, id2seq)
        test_datasets[cell] = test_dataset
    for cell in cell_types:
        test_loader = DataLoader(test_datasets[cell], batch_size=hyperparams['batch_size'])
        test_loaders[cell] = test_loader
print("gathered test")

if load_model:
    print("loading saved model...")
    for cell in train_cells:
        models[train_cells_dict[cell]].load_state_dict(torch.load('./model' + cell + '.pt'))
if train_model:
    for i, cell in enumerate(train_cells):
        print("running training loop", i, "out of", len(train_cells), "for", cell)
        model = models[train_cells_dict[cell]]
        train(model, train_loaders[train_cells_dict[cell]], cell)
        torch.cuda.empty_cache()
        validate(model, validate_loaders[train_cells_dict[cell]])
        torch.cuda.empty_cache()
if save_model:
    print("saving model...")
    for cell in train_cells:
        torch.save(models[train_cells_dict[cell]].state_dict(), './model' + cell + '.pt')
if test_model:
    print("running testing loop...")
    test(models, test_loaders, cell_types)
    

Device cuda
gathering train data cuda
gathered sequences
gathered train
gathered test
running training loop 0 out of 50 for E065
starting train


 53%|███████████████████████████████████████████▏                                     | 81/152 [00:35<00:30,  2.35it/s]

In [ ]:
print("done")